In [1]:

# Model Based approaches: Linear, Exponential, Additive, Multiplicative, Quadratic
# Data driven approaches: AR, MA, ARMA, ARIMA (p-order of auto regressive model, d-order of differencing, q-order of moving avg model)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
walmart = pd.read_csv("footfalls.csv")
walmart.Footfalls.plot()
# Trend = No Linear but quadratic kind of trend is there
# Seasonality = No, Non Stationary data - magnitude of cycle is changing

In [ ]:
walmart.head()
# t - month (1st 2nd 3rd month and so on) - Independent feature
#Footfalls - in Jan 91 how many people visited Walmart and so on - target column
# check slide no. 33 in Forecasting-2 ppt.
# Log of Footfall is needed for Exponential model, t_square is required in Additive seasonality with Quadratic model.
# Jan to Dec are dummy variables for month column.

In [4]:
walmart.shape # 159 months and 17 columns

(159, 17)

In [ ]:
walmart.dtypes

In [ ]:
# to make the model understand about month year column, we need to extract month and year from month column using to_datetime() method
walmart["Date"] = pd.to_datetime(walmart.Month, format="%b-%y") # %b - month, %y - year
walmart

In [ ]:
walmart.dtypes

In [8]:
walmart.shape

(159, 18)

In [9]:
# Extracting Day, Weekday name, month name, year from the date col. using date functions from pandas - only for plotting purpose
walmart["month"] = walmart.Date.dt.strftime("%b") # month extraction
walmart["year"] = walmart.Date.dt.strftime("%y") # year extraction

In [ ]:
walmart.head()

In [ ]:
plt.figure(figsize=(12,3))
sns.lineplot(x="year",y="Footfalls",data=walmart)
# from 91 to 96 footfall was decreasing, then increased till 2003....

# Data Splitting

In [12]:
train = walmart.head(147)
test = walmart.tail(12)

In [ ]:
train

In [ ]:
test

In [15]:
from sklearn.metrics import mean_squared_error

In [17]:
# Linear Model
import statsmodels.formula.api as smf

linear_model = smf.ols('Footfalls~t',data=train).fit() #Footfalls - target col, t - independent variable
pred_linear = pd.Series(linear_model.predict(pd.DataFrame(test['t']))) # get prediction for test data
rmse_linear = np.sqrt(mean_squared_error(test['Footfalls'],pred_linear)) # there is no direct built in function for rmst so take sqrt of MSE
rmse_linear # RMSE should be minimum. try all models. Whichever model will give you min, RMSE that will be our final model


209.92559265462572

In [18]:
# Exponential model

exp = smf.ols('log_footfalls~t',data=train).fit()
pred_exp = pd.Series(exp.predict(pd.DataFrame(test['t']))) # get prediction for test data
rmse_exp = np.sqrt(mean_squared_error(test['Footfalls'],pred_exp)) # there is no direct built in function for rmst so take sqrt of MSE
rmse_exp

2062.9501154673767

In [19]:
# Quadratic Model

quad = smf.ols('Footfalls~t+t_square',data=train).fit()
pred_quad = pd.Series(quad.predict(pd.DataFrame(test[['t','t_square']])))
rmse_quad = np.sqrt(mean_squared_error(test['Footfalls'],pred_quad))
rmse_quad

137.15462741356146

In [20]:
# Additive Seasonality Model - Pass from Jan to Nov

add_sea = smf.ols('Footfalls~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train).fit()
pred_add_sea = pd.Series(add_sea.predict(pd.DataFrame(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']])))
rmse_add_sea = np.sqrt(mean_squared_error(test['Footfalls'],pred_add_sea))
rmse_add_sea

264.6643900568774

In [21]:
# Additive Seasonality Quadratic Model
add_sea_quad = smf.ols('Footfalls~t+t_square+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train).fit()
pred_add_sea_quad = pd.Series(add_sea_quad.predict(pd.DataFrame(test[['t','t_square','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']])))
rmse_add_sea_quad = np.sqrt(mean_squared_error(test['Footfalls'],pred_add_sea_quad))
rmse_add_sea_quad

50.607245841424444

In [22]:
# Multiplicative Seasonality

mul_sea = smf.ols('log_footfalls~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=train).fit()
pred_mul_sea = pd.Series(mul_sea.predict(test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']]))
rmse_mul_sea = np.sqrt(mean_squared_error(test['Footfalls'],pred_mul_sea))
rmse_mul_sea

2062.996088663915

In [ ]:
# Compare the results

data = {"MODEL":pd.Series(['Linear','Exponential','Quadratic','Additive Seas','Additive Seas Quad','Multiplicative Seas']),
        "RMSE_Values":pd.Series([rmse_linear,rmse_exp,rmse_quad,rmse_add_sea,rmse_add_sea_quad,rmse_mul_sea])}
table_rmse=pd.DataFrame(data)
table_rmse.sort_values(["RMSE_Values"])
# RMSE value for additive seasonality with quadratic trend is giving less RMSE value so this model is good for this dataset.
# To predict the footfall for next 12 months additive seasonality with quadratic trend model should be used.

,MODEL,RMSE_Values
4,Additive Seas Quad,50.607246
2,Quadratic,137.154627
0,Linear,209.925593
3,Additive Seas,264.664390
1,Exponential,2062.950115
5,Multiplicative Seas,2062.996089
